In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import panel as pn
import hvplot.xarray
import hvplot.pandas
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
import mysql.connector

pn.extension('tabulator')
pn.extension()
from sqlalchemy import create_engine

# Get dataframes from MySQL database

## helper function

In [2]:
def get_dataframe(table_name):
    con = mysql.connector.connect(host="localhost", database='moodle', user="root", passwd="",use_pure=True)

    cursor = con.cursor()
    query = "SELECT * FROM " + table_name
    cursor.execute(query)

    df = pd.DataFrame(cursor.fetchall())
    df.columns = cursor.column_names

    cursor.close()
    con.close()
    
    return df

## dataframes from database

In [16]:
user = get_dataframe('mdl_user')
quiz = get_dataframe('mdl_quiz')
quiz_attempts = get_dataframe('mdl_quiz_attempts')
quiz_slots = get_dataframe('mdl_quiz_slots')
quiz_grades = get_dataframe('mdl_quiz_grades')
quiz_feedback = get_dataframe('mdl_quiz_feedback')
question = get_dataframe('mdl_question')
question_attempts = get_dataframe('mdl_question_attempts')
question_usages = get_dataframe('mdl_question_usages')
course = get_dataframe('mdl_course')
course_modules = get_dataframe('mdl_course_modules')
course_completion = get_dataframe('mdl_course_completions')
user_enrolments = get_dataframe('mdl_user_enrolments')
enrol = get_dataframe('mdl_enrol')
course_sections = get_dataframe('mdl_course_sections')

# build own database for dashboard

In [26]:
user_data = {'Name': user['firstname'] + ' ' + user['lastname'],
                      'SuS ID': user['id']
                     }

quiz_data = {'Aufgabe': quiz['name'],
             'Quiz ID': quiz['id'],
             'Maximale Punktzahl': quiz['sumgrades'],
             'Maximale Note in %': quiz['grade']
            }

quiz_attempts_data = {'SuS ID': quiz_attempts['userid'],
                      'Quiz ID': quiz_attempts['quiz'],
                      'Frage ID': quiz_attempts['uniqueid'],
                      'Startzeit': quiz_attempts['timestart'],
                      'Abgabezeit': quiz_attempts['timefinish'],
                      'Punktzahl': quiz_attempts['sumgrades'],
                      'Anzahl Versuche': quiz_attempts['attempt']
                     }

quiz_grade_data = {'Beste Punktzahl in %': quiz_grades['grade'],
                   'SuS ID': quiz_grades['userid'],
                   'Quiz ID': quiz_grades['quiz']
                  }

user_enrolments_data = {'Anmelde ID': user_enrolments['enrolid'],
                        'SuS ID': user_enrolments['userid']
                       }

enrol_data = {'Anmelde ID': enrol['id'],
              'Klassen ID': enrol['courseid']
             }

course_data = {'Klasse': course['fullname'],
               'Klassen ID': course['id']
              }

course_sections_data = {'Klassen ID': course_sections['course'],
                        'Fach': course_sections['name']
                       }

df = pd.DataFrame(quiz_data)
df

,Aufgabe,Quiz ID,Maximale Punktzahl,Maximale Note in %
0,Zahlworte,2,6.00000,10.00000
1,Runden,3,180.00000,10.00000
2,Ausklammern,4,400.00000,10.00000
3,Ausklammern Division,5,700.00000,10.00000
4,Ausklammern Multiplikation,6,1200.00000,10.00000
5,Ausklammern mit Dezimalzahlen: Multiplikation,7,600.00000,10.00000
6,Runden,8,9.00000,10.00000


In [28]:
pd.set_option('display.max_rows', None)

df_user = pd.DataFrame(user_data)

df_quiz_attempts = pd.DataFrame(quiz_attempts_data)
df_quiz_attempts['Startzeit'] = pd.to_datetime(df_quiz_attempts['Startzeit'], unit = 's')
df_quiz_attempts['Abgabezeit'] = pd.to_datetime(df_quiz_attempts['Abgabezeit'], unit = 's')
df_quiz_attempts['Bearbeitungsdauer'] = df_quiz_attempts['Abgabezeit'] - df_quiz_attempts['Startzeit']

df_quiz_grade = pd.DataFrame(quiz_grade_data)

df_course_modules = pd.DataFrame(course_modules_data)

df_course = pd.DataFrame(course_data)

df_user_enrolments = pd.DataFrame(user_enrolments_data)

df_enrol = pd.DataFrame(enrol_data)

df_course_sections = pd.DataFrame(course_sections_data)

df_quiz = pd.DataFrame(quiz_data)

df_finished_quiz = pd.merge(df_user, df_quiz_attempts, on = 'SuS ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_quiz_grade, on = ['SuS ID', 'Quiz ID'], how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_user_enrolments, on = 'SuS ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_enrol, on = 'Anmelde ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_course, on = 'Klassen ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_course_sections, on = 'Klassen ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_quiz, on = 'Quiz ID', how = 'left')
df_finished_quiz['Beste Punktzahl in %'] = df_finished_quiz['Beste Punktzahl in %']*10
df_finished_quiz

,Name,SuS ID,Quiz ID,Frage ID,Startzeit,Abgabezeit,Punktzahl,Anzahl Versuche,Bearbeitungsdauer,Beste Punktzahl in %,Anmelde ID,Klassen ID,Klasse,Fach,Aufgabe,Maximale Punktzahl,Maximale Note in %
0,Guest user,1,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,1.0,2.0,Klasse 1a,None,NaN,NaN,NaN
2,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,1.0,2.0,Klasse 1a,Mathematik,NaN,NaN,NaN
3,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,7.0,4.0,Klasse 3b,None,NaN,NaN,NaN
4,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,7.0,4.0,Klasse 3b,Mathematik,NaN,NaN,NaN
5,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,None,NaN,NaN,NaN
6,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN
7,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,Chemie,NaN,NaN,NaN
8,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,None,NaN,NaN,NaN
9,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN


In [29]:
df_finished_quiz

,Name,SuS ID,Quiz ID,Frage ID,Startzeit,Abgabezeit,Punktzahl,Anzahl Versuche,Bearbeitungsdauer,Beste Punktzahl in %,Anmelde ID,Klassen ID,Klasse,Fach,Aufgabe,Maximale Punktzahl,Maximale Note in %
0,Guest user,1,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,1.0,2.0,Klasse 1a,None,NaN,NaN,NaN
2,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,1.0,2.0,Klasse 1a,Mathematik,NaN,NaN,NaN
3,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,7.0,4.0,Klasse 3b,None,NaN,NaN,NaN
4,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,7.0,4.0,Klasse 3b,Mathematik,NaN,NaN,NaN
5,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,None,NaN,NaN,NaN
6,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN
7,Alina Neue,2,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,Chemie,NaN,NaN,NaN
8,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,None,NaN,NaN,NaN
9,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN
